In [66]:
!pip install markdownify duckduckgo-search smolagents --upgrade -q
!pip install -qU langchain-community faiss-cpu
!pip install langchain unstructured python-docx
!pip install markdownify duckduckgo-search spaces gradio-tools langchain langchain-community langchain-huggingface faiss-cpu --upgrade -q



Let's login in order to call the HF Inference API:

# 2. Let's create our multi-agent RAG system

In this section, we will create each of the agents present in our RAG system.

We will have 3 agents managed by a central one (refer to the image for details):

* **🕵💬 Web search agent**: It will include the [`DuckDuckGoSearchTool`](https://github.com/huggingface/transformers/blob/main/src/transformers/agents/search.py) tool and the [`VisitWebpageTool`](https://github.com/huggingface/transformers/blob/main/src/transformers/agents/search.py). As you can see, each agent may contain a list of tools.
* **🕵💬 Retriever agent**: It will include two tools for retrieving information from two different knowledge bases.

In [67]:
import re
import requests
from markdownify import markdownify as md
from requests.exceptions import RequestException
from smolagents import tool


@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = md(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

Ok, now let's initialize and test our tool!

In [ ]:
print(visit_webpage("https://netzero.vn/")[:500])

Trang chủ | NetZero.VN - Net Zero Viet Nam

 

By using this site, you agree to the [Privacy Policy](#) and [Terms of Use](#).

[Accept](#)

[![NetZero.VN - Net Zero Viet Nam](https://netzero.vn/wp-content/uploads/2023/10/netzero_vn.png)![NetZero.VN - Net Zero Viet Nam](https://netzero.vn/wp-content/uploads/2023/10/netzero_vn.png)![NetZero.VN - Net Zero Viet Nam](https://netzero.vn/wp-content/uploads/2023/10/netzero_vn-trans.png)

NetZero.VN - Net Zero Viet Nam

Vi


## Build our multi-agent system 🤖🤝🤖


In [105]:
from smolagents import (
    ToolCallingAgent,
    InferenceClientModel,
    DuckDuckGoSearchTool,
    MultiStepAgent
)

# Khởi tạo model
model = InferenceClientModel("Qwen/Qwen2.5-7B-Instruct")

search_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],  # This is good!
    model=model,
    name="search_agent",
    description="Runs web searches for you. Give it your query as an argument."
)


In [ ]:
result = search_agent(task="net zero là gì vậy?")
print(result)

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'search_agent'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ net zero là gì vậy?                                                                                             │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-7B-Instruct ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'net zero là gì'}                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|Net Zero là gì? Tầm quan trọng và cách để đạt được](https://vnexpress.net/net-zero-la-gi-4794647.html)
Net Zero là gì? Net Zero (phát thải ròng bằng 0) là trạng thái mà tổng lượng khí thải nhà kính trong hoạt động sản 
xuất, kinh doanh là 0. Net Zero là trạng thái mà tổng lượng khí thải nhà kính bao gồm Carbon Dioxide (CO2), Methane
(CH4), Nitrous oxide (N2O), Hydrofluorocarbons (HFCs) bằng 0.

|Net Zero là gì? Sự cam kết phát thải ròng bằng 0 năm 
2050](https://www.pace.edu.vn/tin-kho-tri-thuc/net-zero-la-gi)
Net Zero là một mục tiêu môi trường nhằm giảm lượng phát thải khí nhà kính do con người xuống mức cân bằng với khả 
năng hấp thụ của Trái Đất đến năm 2050. Bài viết này giải thích tầm quan trọng, cơ hội kinh tế và cách đạt được Net
Zero của Việt Nam và thế giới.

|NET ZERO LÀ GÌ...VÀ LÀM THẾ NÀO ĐỂ ĐẠT ĐƯỢC? | Bureau Veritas Việt 
Nam](https://www.bureauveritas.vn/insight/net-zero-la-gi-lam-the-nao-de-dat-duoc)
Net Zero là một mục tiêu tham vọng hơn so với Trung hòa Carbon, đạt được bằng cách cắt giảm lượng khí thải Carbon 
từ các nhà cung cấp đầu tiên đến người dùng cuối. Bài viết này giới thiệu các thuật ngữ liên quan, các khác biệt và
các khó khăn trong việc áp dụng Net Zero.

|Net zero là gì và các nước thực hiện ra sao? - BBC](https://www.bbc.com/vietnamese/world-58969160)
Đạt mục tiêu này là điều không dễ: để đạt net zero, toàn bộ khí thải từ nhà ở của chúng ta, cách chúng ta đi lại và
những gì ta ăn đều bị ảnh ...

|Net zero là gì? Tầm quan trọng của phát thải ròng bằng 0](https://fpt-is.com/goc-nhin-so/net-zero-la-gi/)
Net zero là kết quả của trạng thái cân bằng lượng khí thải nhà kính thải ra khí quyển, mục tiêu của biến đổi khí 
hậu. Bài viết này giải thích tiêu chí net zero, phân biệt net zero và carbon neutral, và các thách thức của việc 
đạt được net zero.

|Net zero là gì? Phát thải ròng về 0 & Tất cả những điều cần 
biết](https://khangducconst.com/knowledge-base/net-zero-la-gi-tat-ca-nhung-dieu-can-biet/)
Net Zero là trạng thái không phát thải carbon dioxide hoặc không thêm vào tổng lượng khí nhà kính thải ra khí 
quyển. Bài viết này giải thích nghĩa, ưu nhược điểm, cách đạt được mục tiêu Net Zero và các khác biệt giữa Net Zero
và năng lượng tái tạo.

|Mục tiêu Net Zero là gì? - som.edu.vn](https://som.edu.vn/muc-tieu-net-zero-la-gi/)
1. Mục tiêu Net Zero là gì? Mục tiêu Net Zero (hay còn gọi là phát thải ròng bằng 0) là cam kết của các tổ chức, 
doanh nghiệp và quốc gia nhằm giảm lượng khí nhà kính (GHG) phát thải xuống mức thấp nhất có thể, đồng thời bù đắp 
phần phát thải còn lại bằng các biện pháp hấp thụ carbon, chẳng hạn như trồng ...

|Net Zero Là Gì? Làm Thế Nào Để Đạt Được Net Zero?](https://renouvo.net/vi/chat-thai-carbon/net-zero-la-gi/)
Net Zero là gì? Theo IPCC, để duy trì mức tăng nhiệt độ toàn cầu ở mức 1.5°C, lượng carbon còn lại (mức phát thải 
còn lại của nhân loại) chỉ ở mức 500 tỷ tấn.

|Net Zero là gì? Vai trò và cách đạt mục tiêu Net Zero](https://alterno.net/vi/net-zero/)
Net Zero là gì? Net Zero hay còn là "phát thải ròng bằng 0" là mục tiêu môi trường nhằm giảm lượng khí nhà kính 
thải ra như CH4, CO2, N2O. Những khí này từ hoạt động sản xuất của con người tạo ra cân bằng với khả năng hấp thụ 
hoặc loại bỏ khí thải của Trái Đất ...

|Net Zero là gì? Vì sao Chính phủ Việt Nam cam kết thực hiện Net 
Zero?](https://longtech.vn/net-zero-la-gi-vi-sao-chinh-phu-viet-nam-cam-ket-thuc-hien-net-zero/)
1. Giới thiệu về Net Zero. Net Zero là gì? Net Zero là thuật ngữ mô tả trạng thái mà lượng khí nhà kính (GHG) được 
thải vào khí quyển bằng lượng khí GHG được loại bỏ. Nói cách khác, Net Zero có nghĩa là không có sự gia tăng tổng 
lượng khí nhà kính trong khí quyển.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Net Zero là trạng thái mà tổng lượng khí thải nhà kính │
│ từ các hoạt động sản xuất và kinh doanh bằng 0. Net Zero (phát thải ròng bằng 0) là một mục tiêu môi trường     │
│ nhằm giảm lượng khí thải nhà kính (bao gồm CO2, CH4, N2O, HFCs) xuống mức cân bằng với khả năng hấp thụ của     │
│ Trái Đất. Đây là trạng thái mà tổng lượng khí thải nhà kính từ các hoạt động sản xuất và kinh doanh bằng 0. Net │
│ Zero không chỉ liên quan đến việc giảm lượng khí thải mà còn bao gồm các biện pháp hấp thụ carbon để bù đắp     │
│ phần phát thải còn lại. Net Zero là một mục tiêu tham vọng hơn so với Trung hòa Carbon, đòi hỏi cắt giảm lượng  │
│ khí thải Carbon từ các nhà cung cấp đầu tiên đến người dùng cuối. Để đạt được mục tiêu Net Zero, các tổ chức,   │
│ doanh nghiệp và quốc gia cần giảm lượng khí thải xuống mức thấp nhất có thể và bù đắp phần phát thải còn lại    │
│ bằng các biện pháp hấp thụ carbon, như trồng cây và sử dụng công nghệ hấp thụ carbon. Đạt mục tiêu Net Zero là  │
│ điều không dễ dàng. Để đạt net zero, toàn bộ khí thải từ nhà ở, phương tiện di chuyển và chế độ ăn uống đều cần │
│ được quản lý và giảm thiểu. Các nước trên thế giới đã đặt mục tiêu Net Zero và đang triển khai các chiến lược   │
│ cụ thể để đạt được mục tiêu này. Việc đạt được Net Zero không chỉ mang ý nghĩa về mặt môi trường mà còn tạo ra  │
│ cơ hội kinh tế và thúc đẩy sự đổi mới công nghệ.'}                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Net Zero là trạng thái mà tổng lượng khí thải nhà kính từ các hoạt động sản xuất và kinh doanh bằng 
0. Net Zero (phát thải ròng bằng 0) là một mục tiêu môi trường nhằm giảm lượng khí thải nhà kính (bao gồm CO2, CH4,
N2O, HFCs) xuống mức cân bằng với khả năng hấp thụ của Trái Đất. Đây là trạng thái mà tổng lượng khí thải nhà kính 
từ các hoạt động sản xuất và kinh doanh bằng 0. Net Zero không chỉ liên quan đến việc giảm lượng khí thải mà còn 
bao gồm các biện pháp hấp thụ carbon để bù đắp phần phát thải còn lại. Net Zero là một mục tiêu tham vọng hơn so 
với Trung hòa Carbon, đòi hỏi cắt giảm lượng khí thải Carbon từ các nhà cung cấp đầu tiên đến người dùng cuối. Để 
đạt được mục tiêu Net Zero, các tổ chức, doanh nghiệp và quốc gia cần giảm lượng khí thải xuống mức thấp nhất có 
thể và bù đắp phần phát thải còn lại bằng các biện pháp hấp thụ carbon, như trồng cây và sử dụng công nghệ hấp thụ 
carbon. Đạt mục tiêu Net Zero là điều không dễ dàng. Để đạt net zero, toàn bộ khí thải từ nhà ở, phương tiện di 
chuyển và chế độ ăn uống đều cần được quản lý và giảm thiểu. Các nước trên thế giới đã đặt mục tiêu Net Zero và 
đang triển khai các chiến lược cụ thể để đạt được mục tiêu này. Việc đạt được Net Zero không chỉ mang ý nghĩa về 
mặt môi trường mà còn tạo ra cơ hội kinh tế và thúc đẩy sự đổi mới công nghệ.

[Step 3: Duration 15.59 seconds| Input tokens: 8,081 | Output tokens: 1,353]

Here is the final answer from your managed agent 'search_agent':
Net Zero là trạng thái mà tổng lượng khí thải nhà kính từ các hoạt động sản xuất và kinh doanh bằng 0. Net Zero (phát thải ròng bằng 0) là một mục tiêu môi trường nhằm giảm lượng khí thải nhà kính (bao gồm CO2, CH4, N2O, HFCs) xuống mức cân bằng với khả năng hấp thụ của Trái Đất. Đây là trạng thái mà tổng lượng khí thải nhà kính từ các hoạt động sản xuất và kinh doanh bằng 0. Net Zero không chỉ liên quan đến việc giảm lượng khí thải mà còn bao gồm các biện pháp hấp thụ carbon để bù đắp phần phát thải còn lại. Net Zero là một mục tiêu tham vọng hơn so với Trung hòa Carbon, đòi hỏi cắt giảm lượng khí thải Carbon từ các nhà cung cấp đầu tiên đến người dùng cuối. Để đạt được mục tiêu Net Zero, các tổ chức, doanh nghiệp và quốc gia cần giảm lượng khí thải xuống mức thấp nhất có thể và bù đắp phần phát thải còn lại bằng các biện pháp hấp thụ carbon, như trồng cây và sử dụng công nghệ hấp thụ carbon. Đạt mục tiêu Net Zero là điề

### 2.2 Retriever agent 🤖🔍

The second agent in our multi-agent system is the **Retriever agent**. This agent is responsible for gathering relevant information from different sources. To achieve this, it will utilize two tools that retrieve data from two separate knowledge bases.

We will reuse two data sources that were previously used in other RAG recipes, which will allow the retriever to efficiently gather information for further processing.

By leveraging these tools, the Retriever agent can access diverse datasets, ensuring a comprehensive collection of relevant information before passing it on to the next step in the system.

Let's explore how to set up the retriever and integrate it into our multi-agent system!


In [70]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
from langchain.document_loaders import UnstructuredWordDocumentLoader

loader = UnstructuredWordDocumentLoader("/content/drive/MyDrive/AgriCarbonDEX.docx")
documents = loader.load()

print(documents[0].page_content)  # In nội dung đầu tiên

AgriCarbonDEX

Bối cảnh

Biến đổi khí hậu và phát thải nhà kính đang là thách thức toàn cầu, đòi hỏi mỗi quốc gia, doanh nghiệp và cá nhân cùng hành động. Tín chỉ carbon và dữ liệu ESG (môi trường – xã hội – quản trị) ngày càng trở thành tài sản có giá trị cao, được giao dịch như một phần của chiến lược giảm phát thải và tài chính bền vững.

Tuy nhiên, thị trường tín chỉ carbon hiện nay đối mặt với các vấn đề lớn:

Khó xác minh nguồn gốc và tính minh bạch của dữ liệu môi trường.

Nguy cơ “greenwashing” khi doanh nghiệp tuyên bố xanh nhưng không có bằng chứng kiểm chứng.

Dữ liệu bị phân mảnh, khó theo dõi xuyên suốt chuỗi giá trị.

Giải thích các khái niệm

DT (Digital Twin): là bản sao kỹ thuật số của một đối tượng, hệ thống hoặc quá trình trong thế giới thực. DT mô phỏng hành vi, trạng thái và hiệu suất của đối tượng thật theo thời gian thực.

Dữ liệu DT: là dữ liệu được thu thập từ cảm biến hoặc nguồn thực tế, phản ánh trạng thái hiện tại hoặc lịch sử của đối tượng.

Tín chỉ carbon:

In [72]:

from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

source_docs = documents


text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=500,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)

print("Embedding documents...")
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
huggingface_doc_vector_db = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

Splitting documents...


100%|██████████| 1/1 [00:00<00:00,  8.40it/s]

Embedding documents...


In [73]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore

class RetrieverTool(Tool):
    name = "retriever"
    description="Truy xuất thông tin từ cơ sở dữ liệu nội bộ"
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.vectordb.similarity_search(
            query,
            k=3,
        )

        return "\nRetrieved documents:\n" + "".join(
            [
                f"===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

In [74]:
in_system_tool = RetrieverTool(huggingface_doc_vector_db)

In [106]:
retriever_agent = ToolCallingAgent(
    tools=[in_system_tool],  # This is good!
    model=model,
    name="retriever_agent",
    description="Agent chuyên tìm kiếm thông tin từ cơ sở tri thức nội bộ về carbon, tín chỉ carbon, dữ liệu tài liệu công ty và các báo cáo PDF.",
)

In [ ]:
retriever_agent.run("AgriCarbonDEX hoạt động như nào ?")

╭─────────────────────────────────────────── New run - retriever_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ AgriCarbonDEX hoạt động như nào ?                                                                               │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-7B-Instruct ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 1: Duration 3.08 seconds]

KeyboardInterrupt: 

In [104]:
from huggingface_hub import notebook_login

notebook_login()

In [92]:
from smolagents import CodeAgent


manager_agent = CodeAgent(
    name="manager_agent",
    description = """
    Tôi là agent điều phối. Tôi sẽ chọn agent phù hợp tùy yêu cầu:
    - Nếu câu hỏi yêu cầu tìm kiếm thông tin có trong tài liệu nội bộ (như tài liệu PDF, dữ liệu RAG), tôi sẽ gọi `retriever_agent`.
    - Nếu yêu cầu liên quan đến việc truy vấn thông tin mới trên internet, tôi sẽ dùng `search_agent`.
    """ ,
    tools=[],
    model=model,
    managed_agents=[search_agent, retriever_agent],
    additional_authorized_imports=["time", "datetime", "PIL"],
)

In [ ]:
manager_agent.run("Trong tài liệu nội bộ, có nói gì về tín chỉ carbon tại Việt Nam không?")

╭──────────────────────────────────────────── New run - manager_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Trong tài liệu nội bộ, có nói gì về tín chỉ carbon tại Việt Nam không?                                          │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-7B-Instruct ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  carbon_credit_info = retriever_agent("tín chỉ carbon tại Việt Nam")                                              
  print(carbon_credit_info)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────── New run - retriever_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'retriever_agent'.                                                                 │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ tín chỉ carbon tại Việt Nam                                                                                     │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-7B-Instruct ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(carbon_credit_info)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Here is the final answer from your managed agent 'retriever_agent':
### 1. Task outcome (short version):
Tín chỉ carbon tại Việt Nam liên quan đến việc xác minh nguồn gốc phát thải và hấp thụ, cũng như việc tạo và giao 
dịch tín chỉ carbon. Nó được ứng dụng trong nhiều lĩnh vực như nông nghiệp tái sinh, trang trại trồng rừng, nhà máy
xanh, tổ chức tài chính, nhà đầu tư ESG, chính quyền địa phương và trung ương, và người dân.

### 2. Task outcome (extremely detailed version):
Tín chỉ carbon tại Việt Nam là một công cụ hữu ích để đảm bảo tính minh bạch và xác thực về nguồn gốc phát thải và 
hấp thụ khí thải nhà kính. Nó giúp tạo ra một thị trường giao dịch tín chỉ carbon minh bạch và phi tập trung, 
khuyến khích doanh nghiệp phát triển bền vững. Một số điểm chính:

- Tín chỉ carbon được tạo ra từ dữ liệu thật, có thể kiểm tra, giúp đảm bảo tính minh bạch.
- Thị trường tín chỉ carbon được xây dựng để tạo ra một hệ thống giao dịch minh bạch, phi tập trung, thúc đẩy phát 
triển bền vững.
- Dữ liệu ESG (môi trường - xã hội - quản trị) được đưa vào tài chính hóa, phục vụ các quỹ đầu tư xanh và nhà hoạch
định chính sách.
- Công nghệ Việt Nam đóng vai trò quan trọng trong việc xây dựng hạ tầng tài chính môi trường.

### 3. Additional context (if relevant):
Việt Nam đã và đang nỗ lực thực hiện các biện pháp nhằm giảm phát thải khí nhà kính và tăng cường sử dụng năng 
lượng tái tạo. Hệ thống tín chỉ carbon tại Việt Nam được xây dựng dựa trên kiến trúc Multi-agent, với các thành 
phần như End devices, Edge, Fog, Control center, và LLM Agent. Hệ thống này sử dụng các cảm biến để thu thập dữ 
liệu từ các trang trại trồng rừng hoặc các xí nghiệp, và sử dụng công nghệ kỹ thuật số để phân tích và quản lý dữ 
liệu.

Tín chỉ carbon tại Việt Nam cũng liên quan đến việc sử dụng Carbon Soundbound Token và Carbon Token. Carbon 
Soundbound Token được cấp cho các tổ chức có ảnh hưởng tiêu cực đến môi trường, trong khi Carbon Token được sử dụng
để mua Credit NFT và bù đắp lượng khí thải carbon của tổ chức.

[Step 2: Duration 29.80 seconds| Input tokens: 5,372 | Output tokens: 432]

"Here is the final answer from your managed agent 'retriever_agent':\n### 1. Task outcome (short version):\nTín chỉ carbon tại Việt Nam liên quan đến việc xác minh nguồn gốc phát thải và hấp thụ, cũng như việc tạo và giao dịch tín chỉ carbon. Nó được ứng dụng trong nhiều lĩnh vực như nông nghiệp tái sinh, trang trại trồng rừng, nhà máy xanh, tổ chức tài chính, nhà đầu tư ESG, chính quyền địa phương và trung ương, và người dân.\n\n### 2. Task outcome (extremely detailed version):\nTín chỉ carbon tại Việt Nam là một công cụ hữu ích để đảm bảo tính minh bạch và xác thực về nguồn gốc phát thải và hấp thụ khí thải nhà kính. Nó giúp tạo ra một thị trường giao dịch tín chỉ carbon minh bạch và phi tập trung, khuyến khích doanh nghiệp phát triển bền vững. Một số điểm chính:\n\n- Tín chỉ carbon được tạo ra từ dữ liệu thật, có thể kiểm tra, giúp đảm bảo tính minh bạch.\n- Thị trường tín chỉ carbon được xây dựng để tạo ra một hệ thống giao dịch minh bạch, phi tập trung, thúc đẩy phát triển bền vữn

In [107]:
import gradio as gr

# Hàm xử lý truy vấn
def ask_multi_agent(question):
    try:
        response = manager_agent.run(question)
        return str(response)
    except Exception as e:
        return f"❌ Lỗi: {str(e)}"

# Tạo giao diện với dark theme
with gr.Blocks(title="Multi-Agent RAG + Web Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🤖 Hỏi hệ thống trợ lý đa tác nhân (Multi-Agent RAG/Web Search)")
    with gr.Row():
        question_input = gr.Textbox(label="💬 Nhập câu hỏi", placeholder="Ví dụ: Tín chỉ carbon là gì?", lines=2)
    with gr.Row():
        output = gr.Textbox(label="📄 Kết quả phản hồi", lines=15)
    with gr.Row():
        submit_btn = gr.Button("🚀 Gửi câu hỏi")

    submit_btn.click(fn=ask_multi_agent, inputs=question_input, outputs=output)

# Chạy giao diện
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e38ed7eda03961a66.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
